In [14]:
import os
import shutil
import fitz  # PyMuPDF
from pathlib import Path

def reemplazar_texto_pdf(archivo_entrada, archivo_salida, reemplazos, tamano_fuente=20):
    """
    Reemplaza texto en un PDF usando un método mejorado para control de tamaño de fuente.
    
    Args:
        archivo_entrada: Ruta al archivo PDF original.
        archivo_salida: Ruta donde guardar el PDF modificado.
        reemplazos: Diccionario con {texto_original: texto_nuevo}.
        tamano_fuente: Tamaño de la fuente para el texto reemplazado.
    
    Returns:
        bool: True si se realizaron cambios, False en caso contrario.
    """
    try:
        # Abrir documento con PyMuPDF
        doc = fitz.open(archivo_entrada)
        cambios_realizados = False
        
        # Procesar cada página
        for num_pagina in range(len(doc)):
            pagina = doc[num_pagina]
            cambios_en_pagina = False
            
            # Buscar y reemplazar cada texto
            for texto_original, texto_nuevo in reemplazos.items():
                # Buscar todas las apariciones del texto original
                instancias = pagina.search_for(texto_original)
                
                if instancias:
                    for inst in instancias:
                        # Inflar el rectángulo manualmente para darle más espacio al texto
                        inst_inflated = fitz.Rect(inst.x0 - 1, inst.y0 - 5, inst.x1 + 5, inst.y1 + 5)
                        # Crear una anotación de redacción para reemplazar el texto
                        annot = pagina.add_redact_annot(
                            inst_inflated,
                            text=texto_nuevo,
                            fontname="helv",      # Helvetica es una fuente estándar
                            fontsize=tamano_fuente  # Tamaño de fuente deseado
                        )
                        annot.update()
                        cambios_en_pagina = True
                    # Aplicar las redacciones para hacer efectivos los cambios
                    pagina.apply_redactions()
                    cambios_realizados = True
            
            if cambios_en_pagina:
                print(f"  - Reemplazos aplicados en página {num_pagina+1}")
        
        # Guardar solo si se realizaron cambios
        if cambios_realizados:
            doc.save(
                archivo_salida, 
                garbage=4,        # Nivel máximo de limpieza
                deflate=True,     # Comprimir flujos
                clean=True,       # Limpiar contenido
                pretty=False      # No formatear para legibilidad (más compacto)
            )
        else:
            doc.close()
            shutil.copy2(archivo_entrada, archivo_salida)
        
        doc.close()
        return cambios_realizados
        
    except Exception as e:
        print(f"Error al reemplazar texto en PDF: {str(e)}")
        shutil.copy2(archivo_entrada, archivo_salida)
        return False

def procesar_pdfs_en_carpeta(ruta_origen, ruta_destino, reemplazos, tamano_fuente=20):
    """
    Procesa todos los PDFs en una carpeta, reemplazando textos y guardando en la carpeta destino.
    
    Args:
        ruta_origen: Carpeta con los PDFs originales.
        ruta_destino: Carpeta donde guardar los PDFs editados.
        reemplazos: Diccionario con {texto_original: texto_nuevo}.
        tamano_fuente: Tamaño de la fuente para el texto reemplazado.
    """
    if not os.path.exists(ruta_destino):
        os.makedirs(ruta_destino)
    
    archivos_pdf = [f for f in os.listdir(ruta_origen) if f.lower().endswith('.pdf')]
    
    total_archivos = len(archivos_pdf)
    archivos_procesados = 0
    archivos_con_cambios = 0
    
    print(f"Encontrados {total_archivos} archivos PDF para procesar.")
    print(f"Usando tamaño de fuente: {tamano_fuente}")
    
    for nombre_archivo in archivos_pdf:
        ruta_completa_origen = os.path.join(ruta_origen, nombre_archivo)
        ruta_completa_destino = os.path.join(ruta_destino, nombre_archivo)
        
        print(f"Procesando: {nombre_archivo}")
        
        try:
            cambios_realizados = reemplazar_texto_pdf(
                ruta_completa_origen,
                ruta_completa_destino,
                reemplazos,
                tamano_fuente
            )
            
            if cambios_realizados:
                archivos_con_cambios += 1
                print(f"✓ Archivo modificado: {nombre_archivo}")
            else:
                print(f"- No se encontraron textos para reemplazar: {nombre_archivo}")
            
            archivos_procesados += 1
            
        except Exception as e:
            print(f"❌ Error al procesar {nombre_archivo}: {str(e)}")
            try:
                shutil.copy2(ruta_completa_origen, ruta_completa_destino)
                print(f"  - Se copió el archivo original debido al error")
            except Exception as copy_error:
                print(f"  - No se pudo copiar el archivo original: {str(copy_error)}")
    
    print(f"\nProceso completado: {archivos_procesados} de {total_archivos} archivos procesados")
    print(f"Archivos modificados: {archivos_con_cambios}")
    print(f"Los archivos se guardaron en: {ruta_destino}")

# Datos de entrada
ruta_origen = r"\\Servernas\AYC2\ASEGURAMIENTO\ASEGURAMIENTO\PROCESO_ASEGURAMIENTO\REGIMEN SUBSIDIADO\MUNICIPIOS 2025\MOVILIDAD_064\01_ENERO\CORREOS USUARIOS"
ruta_destino = r"C:\Users\osmarrincon\Downloads\Unir PDF\Originales"

reemplazos = {
    #"viernes, 7 de febrero de 2025": "jueves, 24 de enero de 2025",
    "Yopal, 15 de enero 2025": "Yopal, 12 de febrero 2025"
}

TAMANO_FUENTE = 20

if __name__ == "__main__":
    print("Iniciando procesamiento de archivos PDF...")
    try:
        procesar_pdfs_en_carpeta(ruta_origen, ruta_destino, reemplazos, TAMANO_FUENTE)
    except Exception as e:
        print(f"Error general en la ejecución: {str(e)}")


Iniciando procesamiento de archivos PDF...
Encontrados 19 archivos PDF para procesar.
Usando tamaño de fuente: 20
Procesando: Correo_CHÁMEZA_Movilidad064_Enero_2025.pdf
  - Reemplazos aplicados en página 1
  - Reemplazos aplicados en página 2
  - Reemplazos aplicados en página 3
  - Reemplazos aplicados en página 4
  - Reemplazos aplicados en página 5
  - Reemplazos aplicados en página 6
  - Reemplazos aplicados en página 7
  - Reemplazos aplicados en página 8
  - Reemplazos aplicados en página 9
  - Reemplazos aplicados en página 10
  - Reemplazos aplicados en página 11
  - Reemplazos aplicados en página 12
  - Reemplazos aplicados en página 13
  - Reemplazos aplicados en página 14
  - Reemplazos aplicados en página 15
  - Reemplazos aplicados en página 16
  - Reemplazos aplicados en página 17
  - Reemplazos aplicados en página 18
  - Reemplazos aplicados en página 19
  - Reemplazos aplicados en página 20
  - Reemplazos aplicados en página 21
  - Reemplazos aplicados en página 22
  - 